In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import re
import time

In [2]:
def scroll(driver, timeout, loops):
    scroll_pause_time = timeout

    last_height = driver.execute_script("return document.body.scrollHeight")
    num = 0
    
    while True and num < loops:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        num = num + 1
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [5]:
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
from time import sleep    

url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1MjQwUEuutHXxVksGEt5qBUDp9DTbssSizNSSxBy1_CTbosSSzLz04vjk_NK8ErV825TU4mS18pLoWFsTAMwwGeo%3D'

profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0")
options = Options()
options.headless = True
browser = webdriver.Firefox(options=options)
options.add_argument("start-maximized")
options.add_argument('disable-infobars')

browser.get(url)
loops = 1
timeout = 2.5
scroll(browser, timeout, loops)

elemlinks = browser.find_elements_by_class_name('anchor__anchor--2QZvA')
links_hrefs = [link.get_attribute('href') for link in elemlinks]

for elem in links_hrefs[72:-1]:
    browser.get(elem)
    scroll(browser, 2, 10)
    print("Pulling from "+str(elem)+"...")

    title = browser.find_element_by_class_name('wine').text
    price = browser.find_element_by_class_name('purchaseAvailability__currentPrice--3mO4u').text.split(" ", 1)[0]
    rating = browser.find_element_by_class_name('vivinoRating__averageValue--3Navj').text
    country = browser.find_element_by_xpath('//a[contains(@href, "/explore?country_code")]').text
    winetype = browser.find_element_by_class_name('wineLocationHeader__wineType--14nrC').text
    
    #browser.implicitly_wait(2)
    #app = browser.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div[1]/div/div[2]')
    
    doClick = False
    try:
        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, r'//*[@id="wine-page-lower-section"]/div[1]/div/div[2]')))    
        doClick = True
    except:
        doClick = False
        
    if(doClick):
        browser.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div[1]/div/div[2]').click()
        
    #WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, r'//*[@id="wine-page-lower-section"]/div[1]/div/div[2]/div')))
    #at = browser.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div[1]/div/div[2]/div').text
    browser.implicitly_wait(1)
    FirstFlag = False
    try:
        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, r'//*[@id="wine-page-lower-section"]/div[1]/div/div[2]/div')))
        unfilteredText = browser.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div[1]/div/div[2]/div').text
        print(unfilteredText)
        flag = True
    except:
        flag = False
    
    SecFlag = False
    try:
        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, r'//*[@id="wine-page-lower-section"]/div[1]/div/div[2]/div/div[2]/div[1]/table/tbody/tr[1]/td[2]/div/span')))
        SecFlag = True
    except:
        SecFlag = False
    
    #WebDriverWait(browser, 20).until(EC.presence
    
    temp = browser.find_element_by_class_name('vintage')
    year = 9999
    line = re.findall(r'\d+', temp.text)
    if(len(line)>0):
        for l in line:
            if(len(str(l)) == 4):
                year = l
                
    shavedbold = 0; shavedsweet = 0; shavedacidic = 0
    #print(browser.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div[1]/div/div[2]/div').text)
    if(SecFlag == True):
        LightOrBold = browser.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div[1]/div/div[2]/div/div[2]/div[1]/table/tbody/tr[1]/td[2]/div/span')
        DryOrSweet = browser.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div[1]/div/div[2]/div/div[2]/div[1]/table/tbody/tr[2]/td[2]/div/span')
        SoftOrAcidic = browser.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div[1]/div/div[2]/div/div[2]/div[1]/table/tbody/tr[3]/td[2]/div/span')
        shavedbold = LightOrBold.get_attribute("style").split()[3][:-2]
        if shavedbold=='0p':
            shavedbold='0.0001'
        shavedsweet = DryOrSweet.get_attribute("style").split()[3][:-2]
        if shavedsweet=='0p':
            shavedsweet='0.0001'
        shavedacidic = SoftOrAcidic.get_attribute("style").split()[3][:-2]
        if shavedacidic=='0p':
            shavedacidic='0.0001'
        #print(shavedbold)
    
    with open('tempRemove.csv', 'a', encoding='UTF8') as f:
        writer = csv.writer(f)
                    
        data = [title, year, price, rating, country, winetype, shavedbold, shavedsweet, shavedacidic, elem]
        writer.writerow(data)

In [4]:
df = pd.read_csv('vinDataWithNotes.csv')
df.head()

EmptyDataError: No columns to parse from file

In [6]:
df = pd.read_csv('tempRemove.csv')
df.head()

,Name,Year,Price,Rating,Country,Winetype,Boldness,Sweetness,Acidity,Link
0,Terre des Anges Rosé,2019,149.00,4.1,France,Rosé wine,30.8179,1.72861,68.4358,https://www.vivino.com/DK/en/fr-chateau-paradi...
1,Château de Berne Rosé,2019,209.95,4.1,France,Rosé wine,32.2581,0.00010,65.9022,https://www.vivino.com/DK/en/berne-chateau-de-...
2,Finca El Bosque Rosado de Lágrima,2019,110.00,3.4,Spain,Rosé wine,0.0000,0.00000,0.0000,https://www.vivino.com/DK/en/bodegas-ochoa-fin...
3,Belle Année Rosé,2019,59.00,3.7,France,Rosé wine,0.0000,0.00000,0.0000,https://www.vivino.com/DK/en/mirabeau-belle-an...
4,Rosé,2017,69.00,3.1,South Africa,Rosé wine,0.0000,0.00000,0.0000,https://www.vivino.com/DK/en/marvelous-rose/w/...
